In [16]:
!rm -rf hugeCTR_saved_model_MF
!mkdir hugeCTR_saved_model_MF

config = """{
  "solver": {
    "lr_policy": "fixed",
    "display": 1000,
    "max_iter":50000,
    "gpu": [0],
    "batchsize": 16384,
    "snapshot": 10000,
    "snapshot_prefix": "./hugeCTR_saved_model_MF/",
    "eval_interval": 5000,
    "eval_batches": 1000,
    "mixed_precision": 1024,
    "eval_metrics": ["AUC:1.0"]
  },
  
  "optimizer": {
    "type": "SGD",
    "global_update": false,
    "sgd_hparam": {
    "learning_rate": 0.001,
    "warmup_steps": 1000,
     "decay_start": 10000,
    "decay_steps": 10000,
    "end_lr": 0.001
    }
  },


  "layers": [ 
      {
      "name": "data",
      "type": "Data",
      "slot_size_array": [138493 , 26744],
      "slot_size_array_orig": [138493 , 26744],
      "source": "./data/hugeCTR/filelist.txt",
      "eval_source": "./data/hugeCTR/test_filelist.txt",
      "check": "None",
      "cache_eval_data": true,
      "label": {
              "top": "label",
              "label_dim": 1
      },
      "dense": {
              "top": "dense",
              "dense_dim": 1
      },
      "sparse": [
              {
          "top": "data1",
          "type": "LocalizedSlot",
          "max_feature_num_per_sample": 2,
          "max_nnz": 1,
          "slot_num": 2
              }
      ]
    },

    {
      "name": "sparse_embedding1",
      "type": "LocalizedSlotSparseEmbeddingOneHot",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "slot_size_array": [138493 , 26744],
        "embedding_vec_size": 64,
        "combiner": 0
      }
    },

    {
      "name": "reshape1",
      "type": "Reshape",
      "bottom": "sparse_embedding1",
      "top": "reshape1",
      "leading_dim": 64
    },
    
    {
      "name": "slice1",
      "type": "Slice",
      "bottom": "reshape1",
      "ranges": [[0,1], [1,2]],
      "top": ["user", "item"]
    },
      
    {
      "name": "concat1",
      "type": "Concat",
      "bottom": ["user","item"],
      "top": "concat1"
    },

    {
      "name": "fc4",
      "type": "FusedInnerProduct",
      "bottom": "concat1",
      "top": "fc4",
       "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc5",
      "type": "FusedInnerProduct",
      "bottom": "fc4",
      "top": "fc5",
      "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc6",
      "type": "FusedInnerProduct",
      "bottom": "fc5",
      "top": "fc6",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc7",
      "type": "FusedInnerProduct",
      "bottom": "fc6",
      "top": "fc7",
      "fc_param": {
        "num_output": 256
      }
    },

    {
      "name": "fc8",
      "type": "InnerProduct",
      "bottom": "fc7",
      "top": "fc8",
      "fc_param": {
        "num_output": 1
      }
    },
    
    {
      "name": "loss",
      "type": "BinaryCrossEntropyLoss",
      "bottom": ["fc8","label"],
      "top": "loss"
    } 
  ]
}
"""

In [17]:
with open('./movie_lens_MF.json', 'wt') as f:
    for line in config:
        f.write(line)

In [18]:
!../build/bin/huge_ctr --train ./movie_lens_MF.json

[0, init_start, ]
HugeCTR Version: 2.2
Config file: ./movie_lens_MF.json
[11d06h43m21s][HUGECTR][INFO]: batchsize_eval is not specified using default: 16384
Mixed Precision training with scaler: 1024 is enabled.
[11d06h43m21s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[11d06h43m21s][HUGECTR][INFO]: Algorithm search: ON
[11d06h43m22s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[11d06h43m22s][HUGECTR][INFO]: Initial seed is 253050631
[11d06h43m22s][HUGECTR][INFO]: num_internal_buffers 1
[11d06h43m22s][HUGECTR][INFO]: num_internal_buffers 1
[11d06h43m22s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=165237
[11d06h43m51s][HUGECTR][INFO]: gpu0 start to init embedding of slot0 , slot_size=138493, key_offset=0, value_index_offset=0
[11d06h43m52s][HUGECTR][INFO]: gpu0 start to init embedding of slot1 , slot_size=26744, key_offset=138493, value_index_offset=138493
[11d06h43m52s][HUGECTR][INFO]: decay_power is not specifie